In [225]:
#!/Users/iiyakhruschev/opt/anaconda3/bin python
# coding: utf-8

import pandas as pd
import os
import translators as ts
from datetime import datetime
from langdetect import detect
from json_parsers import *
from sqlalchemy import create_engine
from pymongo import MongoClient

# logging
logging.basicConfig(filename='logs/etl_{}'.format(datetime.now()), level=logging.INFO)
# Mongo database and collection
client_db = 'stream'
client_col = 'frenchgp'  # args.collectionc

# source and target connections
MONGO_URI = os.getenv("MONGO_URI")
client = MongoClient(MONGO_URI)

# database and collection names
db = client[client_db]
col = db[client_col]

# IMPORT
df_in = pd.DataFrame(list(col.find({})))
df_in = df_in.drop(['matching_rules'], 1)
# logging.info('{} - {} records retrieved from mongo'.format(datetime.now(), len(df_in)))


# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ DEFINE FUNCTIONS ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
def getData(df):
    """
    Convert the nexted json inside the data column to its own dataframe
    """
    df['author_id'] = df.apply(
        lambda df: parse_json(df, 'data', 'author_id'), 1)
    df['created_at'] = df.apply(
        lambda df: parse_json(df, 'data', 'created_at'), 1)
    df['geo'] = df.apply(lambda df: parse_json(df, 'data', 'geo'), 1)
    df['tweet_id'] = df.apply(lambda df: parse_json_exact(df, 'data', 'id'), 1)
    df['raw_text'] = df.apply(lambda df: parse_json(df, 'data', 'text'), 1)
    df = df[['tweet_id', 'author_id', 'created_at', 'raw_text']]
    return df


def getTranslation(df):
    clean_text = df['clean_text']
    try:
        translated_text = ts.google(clean_text, if_use_cn_host=True)
    except Exception:
        translated_text = clean_text
    return translated_text


def getCleanText(df):
    clean_text = ''.join(e for e in df['raw_text'] if e.isascii())
    clean_text = ''.join(e for e in clean_text if e not in ["!", "@", "#"])
    return clean_text

def getRawText(df, column):
    raw_text = ''.join(e for e in df[column] if e.isascii())
    return raw_text


def getLanguage(df):
    clean_text = ''.join(e for e in df['raw_text'] if e.isascii())
    clean_text = ''.join(e for e in clean_text if e not in ["!", "@", "#"])
    try:
        language = detect(clean_text)
    except Exception:
        language = ''
    return language


def getUsers(df):
    users = df['includes']['users']
    return users


def getUserDataframe(df):
    df['user_created_at'] = df.apply(lambda df: parse_json(df, 'users', 'created_at'), 1)
    df['user_id'] = df.apply(lambda df: parse_json(df, 'users', 'id'), 1)
    df['location'] = df.apply(lambda df: parse_json(df, 'users', 'location'), 1)
    df['name'] = df.apply(lambda df: parse_json_exact(df, 'users', 'name'), 1)
    df['username'] = df.apply(lambda df: parse_json(df, 'users', 'username'), 1)
    df = df.drop(['users'], 1)
    return df


def escapeArray(df, column):
    if len(df[column]) == 0:
        return ''
    else:
        return df[column]

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ CLEAN TWEET DATA / TRANSLATE ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ 
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

data_df = getData(df_in)

In [226]:
for column in list(data_df):
    data_df[column] = data_df.apply(lambda data_df: escapeArray(data_df, column), 1)

for i,j in zip(data_df['tweet_id'], range(len(data_df))):
    if isinstance(i, np.ndarray):
        data_df['tweet_id'][j] = data_df['tweet_id'][j][0]

data_df['clean_text'] = data_df.apply(lambda data_df: getCleanText(data_df), 1)
logging.info('{} - text cleaned'.format(datetime.now()))

data_df['raw_text'] = data_df.apply(lambda data_df: getRawText(data_df, 'raw_text'), 1)
logging.info('{} - emojis removed from raw text'.format(datetime.now()))

data_df['language'] = data_df.apply(lambda data_df: getLanguage(data_df), 1)
logging.info('{} - languages identified '.format(datetime.now()))

data_df = data_df[data_df['tweet_id'] != ''].reset_index()

data_df_translate = data_df[data_df['language'] != 'en'][['tweet_id', 'clean_text']]

/Users/iiyakhruschev/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/iiyakhruschev/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3343: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/iiyakhruschev/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


In [ ]:
logging.info('{} - {} non-english language records detected'.format(datetime.now(), len(data_df_translate)))
data_df_translate['translated_text'] = data_df_translate.apply(lambda data_df_translate: getTranslation(data_df_translate), 1)
logging.info('{} - {} non-english language records translated'.format(datetime.now(), len(data_df_translate)))

data_df_translate['tweet_id'] = data_df_translate['tweet_id'].astype(str)
data_df['tweet_id'] = data_df['tweet_id'].astype(str)

data_df = data_df.merge(data_df_translate, on='tweet_id', how='left')\
    .rename(index=str, columns={'clean_text_x': 'clean_text'})\
    .drop('clean_text_y', 1)

data_df['translated_text'] = data_df['translated_text'].fillna(data_df['clean_text'])
data_df = data_df[['tweet_id', 'author_id', 'created_at',
                   'language', 'raw_text', 'clean_text', 'translated_text']]


# SEND TO POSTGRES
logging.info('{} - sending tweet table to postgres'.format(datetime.now()))
engine = create_engine('postgresql://postgres@localhost:5432/frenchgp')
data_df.to_sql('tweet', engine)
logging.info('{} - export successful'.format(datetime.now()))

In [213]:
df_in['includes'] = df_in['includes'].fillna('')
df_in = df_in[df_in['includes'] != '']

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ USERS MENTIONED ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ 
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


def getUsers(df):
    users = df['includes']['users']
    return users


df_in['users'] = df_in.apply(lambda df_in: getUsers(df_in), 1)

users_exploded = df_in[['tweet_id', 'users']].explode('users')
users_df = getUserDataframe(users_exploded)
users_df['location'] = users_df.apply(
    lambda users_df: escapeArray(users_df, 'location'), 1)

In [214]:
users_df['name'] = users_df.apply(lambda users_df: getRawText(users_df, 'name'), 1)

In [228]:
users_df = users_df.reset_index(drop=True)

In [230]:
for column in list(users_df):
    users_df[column] = users_df.apply(lambda users_df: escapeArray(users_df, column), 1)

for i,j in zip(users_df['tweet_id'], range(len(users_df))):
    if isinstance(i, np.ndarray):
        users_df['tweet_id'][j] = users_df['tweet_id'][j][0]

In [232]:
# SEND TO POSTGRES
# logging.info('{} - sending users mentioned table to postgres'.format(datetime.now()))
users_df.to_sql('users_mentioned', engine)
# logging.info('{} - export successful'.format(datetime.now()))
# logging.info('{} - ETL COMPLETE'.format(datetime.now()))